**STEP 1 — Import required libraries**

In [3]:
import sqlite3
import pandas as pd

**STEP 2 — Connect to the SQLite database**

In [6]:
conn = sqlite3.connect("../sql/supply_chain.db")

**STEP 3 — Write your FIRST SQL query**

In [9]:
query = """
SELECT
    "Order Region" AS order_region,
    "Shipping Mode" AS shipping_mode,
    COUNT(*) AS total_orders,
    SUM(is_delayed) AS delayed_orders,
    ROUND(100.0 * SUM(is_delayed) / COUNT(*), 2) AS delay_percentage
FROM orders_fact
GROUP BY "Order Region", "Shipping Mode"
ORDER BY delay_percentage DESC;
"""

**STEP 4 — Execute the SQL query**

In [12]:
delay_kpi = pd.read_sql(query, conn)

**STEP 5 — Close database connection**

In [15]:
conn.close()

In [17]:
delay_kpi.head(10)

,order_region,shipping_mode,total_orders,delayed_orders,delay_percentage
0,Canada,First Class,184,184,100.0
1,Caribbean,First Class,1193,1193,100.0
2,Central Africa,First Class,300,300,100.0
3,Central America,First Class,4382,4382,100.0
4,Central Asia,First Class,63,63,100.0
5,East Africa,First Class,280,280,100.0
6,East of USA,First Class,1089,1089,100.0
7,Eastern Asia,First Class,1043,1043,100.0
8,Eastern Europe,First Class,658,658,100.0
9,North Africa,First Class,486,486,100.0


**STEP 7 — Sanity checks**

In [20]:
delay_kpi.shape

(92, 5)

In [22]:
delay_kpi.describe()

,total_orders,delayed_orders,delay_percentage
count,92.000000,92.000000,92.000000
mean,1962.163043,1123.913043,66.245761
std,2825.295119,1308.992949,25.596383
min,33.000000,11.000000,20.310000
25%,399.250000,207.250000,40.715000
50%,1066.000000,731.000000,72.665000
75%,2094.250000,1429.750000,92.950000
max,17055.000000,6853.000000,100.000000


### **“Is First Class uniquely bad, or are others also risky?”**

In [26]:
query = """
SELECT
    "Shipping Mode" AS shipping_mode,
    COUNT(*) AS total_orders,
    SUM(is_delayed) AS delayed_orders,
    ROUND(100.0 * SUM(is_delayed) / COUNT(*), 2) AS delay_percentage
FROM orders_fact
GROUP BY "Shipping Mode"
ORDER BY delay_percentage DESC;
"""

In [28]:
conn = sqlite3.connect("../sql/supply_chain.db")
shipping_mode_kpi = pd.read_sql(query, conn)
conn.close()

shipping_mode_kpi

,shipping_mode,total_orders,delayed_orders,delay_percentage
0,First Class,27814,27814,100.00
1,Second Class,35216,28078,79.73
2,Same Day,9737,4657,47.83
3,Standard Class,107752,42851,39.77


## **Time-based Delay Analysis (MONTHLY TREND)**
* keep shipping mode fixed

* analyze delay trend over time

* verify if First Class is always bad or only sometimes bad

**Step 1: Create a month column**

In [37]:
conn = sqlite3.connect("../sql/supply_chain.db")

query = """
SELECT
    strftime('%Y-%m', "order date (DateOrders)") AS order_month,
    "Shipping Mode" AS shipping_mode,
    COUNT(*) AS total_orders,
    SUM(is_delayed) AS delayed_orders,
    ROUND(100.0 * SUM(is_delayed) / COUNT(*), 2) AS delay_percentage
FROM orders_fact
GROUP BY order_month, shipping_mode
ORDER BY order_month, delay_percentage DESC;
"""

monthly_delay = pd.read_sql(query, conn)
conn.close()

monthly_delay.head(10)

,order_month,shipping_mode,total_orders,delayed_orders,delay_percentage
0,2015-01,First Class,710,710,100.00
1,2015-01,Second Class,1095,892,81.46
2,2015-01,Same Day,258,108,41.86
3,2015-01,Standard Class,3329,1345,40.40
4,2015-02,First Class,769,769,100.00
5,2015-02,Second Class,961,764,79.50
6,2015-02,Same Day,264,137,51.89
7,2015-02,Standard Class,2821,1034,36.65
8,2015-03,First Class,801,801,100.00
9,2015-03,Second Class,1101,855,77.66


**Step 2: Sanity checks**

In [40]:
monthly_delay["order_month"].nunique()

48

In [42]:
monthly_delay.groupby("shipping_mode")["delay_percentage"].describe()

,count,mean,std,min,25%,50%,75%,max
shipping_mode,,,,,,,,
First Class,48.0,100.000000,0.000000,100.00,100.0000,100.000,100.0000,100.00
Same Day,48.0,48.401250,21.504828,0.00,39.0225,49.000,57.3075,100.00
Second Class,48.0,80.914167,5.840194,62.50,78.8050,79.925,81.6650,100.00
Standard Class,48.0,39.821042,2.262284,34.21,38.6000,39.675,41.1000,48.57


### **Product Risk**
Why?

* Products tie directly to revenue

* Easier to explain in interviews

* Leads naturally to vendor / sourcing discussions

In [46]:
query = """
SELECT
    "Category Name" AS category,
    "Shipping Mode" AS shipping_mode,
    COUNT(*) AS total_orders,
    SUM(is_delayed) AS delayed_orders,
    ROUND(100.0 * SUM(is_delayed) / COUNT(*), 2) AS delay_percentage
FROM orders_fact
GROUP BY category, shipping_mode
HAVING COUNT(*) > 100
ORDER BY delay_percentage DESC;
"""

conn = sqlite3.connect("../sql/supply_chain.db")
category_shipping_risk = pd.read_sql(query, conn)
conn.close()

category_shipping_risk.head(10)

,category,shipping_mode,total_orders,delayed_orders,delay_percentage
0,Accessories,First Class,290,290,100.0
1,Baseball & Softball,First Class,105,105,100.0
2,Cameras,First Class,152,152,100.0
3,Camping & Hiking,First Class,2057,2057,100.0
4,Cardio Equipment,First Class,1923,1923,100.0
5,Cleats,First Class,3745,3745,100.0
6,Electronics,First Class,499,499,100.0
7,Fishing,First Class,2715,2715,100.0
8,Girls' Apparel,First Class,200,200,100.0
9,Golf Balls,First Class,209,209,100.0


### **Question we now answer:**

**“Among NON–First Class shipments, which categories are actually risky?”**

**Step 1 — Filter out First Class (in SQL)**

In [52]:
query = """
SELECT
    "Category Name" AS category,
    "Shipping Mode" AS shipping_mode,
    COUNT(*) AS total_orders,
    SUM(is_delayed) AS delayed_orders,
    ROUND(100.0 * SUM(is_delayed) / COUNT(*), 2) AS delay_percentage
FROM orders_fact
WHERE "Shipping Mode" != 'First Class'
GROUP BY category, shipping_mode
HAVING COUNT(*) > 200
ORDER BY delay_percentage DESC;
"""

conn = sqlite3.connect("../sql/supply_chain.db")
non_first_class_risk = pd.read_sql(query, conn)
conn.close()

non_first_class_risk.head(10)

,category,shipping_mode,total_orders,delayed_orders,delay_percentage
0,Golf Balls,Second Class,277,231,83.39
1,Golf Gloves,Second Class,211,175,82.94
2,Trade-In,Second Class,214,174,81.31
3,Electronics,Second Class,599,485,80.97
4,Accessories,Second Class,360,291,80.83
5,Girls' Apparel,Second Class,229,185,80.79
6,Shop By Sport,Second Class,2157,1728,80.11
7,Cleats,Second Class,4816,3855,80.05
8,Fishing,Second Class,3314,2650,79.96
9,Indoor/Outdoor Games,Second Class,3750,2991,79.76


**Step 2 — What you should EXPECT now**

You should see:

* Variation across categories

* Some categories worse in Second Class

* Some unstable in Same Day

* Standard Class mostly stable

### **REGION RISK**
The correct question now:

* **“Within NON–First Class shipments, which regions are most unreliable?”**

In [62]:
query = """
SELECT
    "Order Region" AS region,
    "Shipping Mode" AS shipping_mode,
    COUNT(*) AS total_orders,
    SUM(is_delayed) AS delayed_orders,
    ROUND(100.0 * SUM(is_delayed) / COUNT(*), 2) AS delay_percentage
FROM orders_fact
WHERE "Shipping Mode" != 'First Class'
GROUP BY region, shipping_mode
HAVING COUNT(*) > 300
ORDER BY delay_percentage DESC;
"""

conn = sqlite3.connect("../sql/supply_chain.db")
region_risk = pd.read_sql(query, conn)
conn.close()

region_risk.head(10)

,region,shipping_mode,total_orders,delayed_orders,delay_percentage
0,Eastern Asia,Second Class,1406,1178,83.78
1,South of USA,Second Class,829,683,82.39
2,Caribbean,Second Class,1631,1336,81.91
3,North Africa,Second Class,617,504,81.69
4,Western Europe,Second Class,5438,4412,81.13
5,West of USA,Second Class,1632,1313,80.45
6,Southeast Asia,Second Class,1833,1465,79.92
7,South Asia,Second Class,1552,1240,79.90
8,West Africa,Second Class,738,588,79.67
9,Southern Europe,Second Class,1698,1347,79.33


## **1️⃣ Final analytical conclusions**

### **A. Shipping mode is the dominant risk driver**

You’ve proven this multiple times:

* First Class → 100% delay always

* Second Class → ~80–84% delay across regions

* Same Day → volatile

* Standard Class → most stable

This is structural, not accidental.

### **B. Region is a secondary amplifier, not the root cause**

Look at your region results: <br>
* Eastern Asia → 83.78% <br>
* South of USA → 82.39% <br>
* Caribbean → 81.91% <br>
* Western Europe → 81.13%

**Now look at the spread:**

* Max ≈ 83.8%

* Min ≈ 79.3%

* Difference ≈ 4–5%

That is not a game-changer.

**Meaning:**

* Regions differ slightly

* But region does NOT overturn shipping mode behavior

This is an important negative finding.

### **C. Category is irrelevant once shipping mode is fixed**

You already proved:

* Categories cluster tightly around ~80%

* No category stands out meaningfully

So:

Category-based operational intervention will not fix delays.